### Imprort libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import scipy.signal as sps
from IPython.display import HTML

### Functions Definition

In [ ]:
def alias_option(signal_name, sampling_points):
    res = 0
    if signal_name == 'alias_Chirp':
        res = sps.chirp(sampling_points, 0, 1, 20)
        
    elif signal_name == 'alias_Cosine':
        res = np.cos(20*np.pi*sampling_points)
    
    elif signal_name == 'alias_Square':
        res = (np.mod(sampling_points,0.2)>0.1)*1
        
    return res

### Signal Model Initialisation

In [ ]:
# We use a Hamming window filter with a transition band of
# 6.6*pi/M.  If we assume a 1% transition band, then
# M>=330.  As we want a delay of a whole number of samples,
# an odd length is required
flength = 331
samplingInterval = 0.004

# Define the sampling rate for the "original" signal
original_samplingInterval = 0.002

signal_options = ['Cosine', 'Chirp', 'Square']
signal = 'alias_' + signal_options[0]

# Iteration interval of animation, between 0,1
step = 0.005

### Animation

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))
plt.rcParams.update({'font.size': 16})
matplotlib.rcParams['animation.embed_limit'] = 23

ax.set_xlabel('Time', fontsize=16)
ax.set_ylabel('Amplitude', fontsize=16)
ax.set_xlim(0, 1)

reconstruct, = ax.plot([], [], 'g-', linewidth = 1.3, label = 'Reconstructed Signal')
originalSignal, = ax.plot([], [],'b-', linewidth = 1.3,label = 'Original Signal')

(markerLines, stemLines, baseLines) = ax.stem([0], [0], label = 'Sampled Signal')
plt.setp(stemLines, color = 'red', linewidth=1.3) 
plt.setp(baseLines, color = 'black', linewidth=1.3)
plt.setp(markerLines, color = 'red') 
markerLines.set_markerfacecolor('none')

ax.legend(loc = 'upper right')

def animate(i):
    value = (i+1) * step
    samplingInterval = 0.004 + value*0.096
    
    ## Perform the sampling
    # Determine the ratio of downsampling (non-integer)
    sampling_ratio = samplingInterval / original_samplingInterval
    
    # We assume that the reconstructed sampling rate is
    # approximately the same regardless of the downsampling rate.
    # So the filter transition band will not change significantly,
    # meaning that the filter length is fixed
    pre_post_filter = (flength-1) / 2
    
    # Determine the number of extra samples required before the
    # start and after the end to allow for the filter tails.  This
    # should be a whole number of samples, and be as long as the
    # half filter length
    extra_samples = np.ceil(pre_post_filter / sampling_ratio)
    
    # Generate the vector of time values corresponding to the
    # samples
    sampling_points = np.arange(-extra_samples*samplingInterval,
                                1 + extra_samples*samplingInterval + samplingInterval/2,
                                samplingInterval)

    # Compute the sampled values at these points
    sampled_values = alias_option(signal, sampling_points)
    
    ## Generate the reconstructed signal
    # Choose an interpolation factor that results in a sample
    # spacing no larger than the original sampling
    interp_factor = np.ceil(sampling_ratio)

    # Define the FIR filter to remove all images above half of the
    # sampling rate (note that fir1 works in a normalised frequency
    # where 1 is the Nyquist rate).  The filter is scaled as this
    # is an interpolating filter
    h = interp_factor * sps.firwin(numtaps = flength-1,
                                 cutoff = 1/interp_factor, 
                                 pass_zero = True)

    # Create a vector for the interpolated signal prior to filtering
    interp_wz = np.zeros(int(len(sampled_values)*interp_factor))

    # and copy the sampled values into this
    interp_wz[np.arange(0, len(interp_wz), interp_factor, dtype = int)] = sampled_values

    # Applying the filter results in a smoothed interpolated signal
    interpolated = sps.lfilter(h, [1], interp_wz)
    
    ## Plot the results
    # Compute the offset which comprises the samples prior to time
    # 0, plus the filter delay (half of the filter length)
    offset = interp_factor*extra_samples + pre_post_filter

    # Extract the signal corresponding to the original time range
    reconstructed = interpolated[np.arange(0, int(np.ceil(interp_factor/samplingInterval))) 
                                 + 1 + int(offset)]

    # and the time vector associated with these samples
    rt = (np.arange(0, int(np.ceil(interp_factor/samplingInterval)))
          * samplingInterval / interp_factor)
    
    
    ## Reconstructed Signal plot
    reconstruct.set_data(rt, reconstructed)
    

    ## Original Signal plot
    t = np.arange(0, 1.002, original_samplingInterval)
    s = alias_option(signal, t)
    
    ymin = min(s)
    ymax = max(s)
    YLim = np.array([ymin, ymax]) + (ymax-ymin)*np.array([-0.15, 0.15])
    ax.set_ylim(YLim)
    
    originalSignal.set_data(t, s)
    
    
    ## update stem plot, need clear operation cla()

    markerLines.set_data(sampling_points, sampled_values)
    baseLines.set_data([0, len(sampling_points)], [0,0])
    stem_len = len(stemLines)
    
    for i in range(len(sampling_points)):
        if i < stem_len:
            stemLines[i].set_data([sampling_points[i], sampling_points[i]], [0, sampled_values[i]])
        else:
            stemLines.append(matplotlib.lines.Line2D([sampling_points[i], sampling_points[i]],
                                                     [0, sampled_values[i]], color = "red"))
            ax.add_line(stemLines[i])

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=int(1/step))
plt.close()

HTML(ani.to_jshtml())

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.